In [13]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [14]:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 20
       }
matplotlib.rc('font', **font)

In [15]:
from os import chdir

chdir("/home/jaimedgp/Repositories/AirQualityCOVID/")
variable = "no2"

In [18]:
data = pd.read_csv("data/Results/predictions_municipios.csv")
data = data.loc[data["variable"] == variable]

data["date"] = pd.to_datetime(data["date"])

In [19]:
data.head()

,Municipio,variable,date,obs,pred,diff
366,Alcalá de Henares,no2,2020-01-01,42.750000,60.352493,-29.166141
367,Alcalá de Henares,no2,2020-01-02,52.666667,59.055524,-10.818390
368,Alcalá de Henares,no2,2020-01-03,46.750000,52.506599,-10.963572
369,Alcalá de Henares,no2,2020-01-04,44.166667,42.457306,4.026071
370,Alcalá de Henares,no2,2020-01-05,43.875000,56.306616,-22.078429


In [20]:
a = pd.pivot_table(data, 
                   columns=["Municipio"], 
                   index=["date"]
                  ).drop([("diff", "Marbella"),
                          #("diff", "Almería"),
                         ], axis=1)

In [21]:
a = (a - a.min()) / (a.max() - a.min())

In [23]:
def replace_nan(time_serie):
    #for i in index:
    time_serie.interpolate(limit_direction="both",inplace=True)
    
    return time_serie
    
a = a.apply(replace_nan).droplevel(None, axis=1)

In [24]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
from sklearn.preprocessing import MinMaxScaler

/home/jaimedgp/.local/lib/python3.8/site-packages/tslearn/clustering/kmeans.py:16: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  warnings.warn(


In [25]:
q = to_time_series_dataset(a.T.to_numpy(
                  ).reshape(a.shape[1], a.shape[0], 1))

```python
from tslearn.barycenters import softdtw_barycenter, dtw_barycenter_averaging

means = pd.DataFrame(data={"mean":a.mean(axis=1).values, 
                           "softdtw":softdtw_barycenter(q).ravel(), 
                           "dtw":dtw_barycenter_averaging(q).ravel()})

means.plot()
```

In [26]:
seed = 0
np.random.seed(seed)
median_serie = pd.DataFrame()
#print(median_serie.head())

for j in [4]:
    model = TimeSeriesKMeans(n_clusters=j, 
                             metric="euclidean",
                             verbose=False,
                             max_iter_barycenter=5,
                             n_jobs = 6,
                             random_state=seed
                            )

    result = model.fit_predict(q)

    fig=plt.figure(figsize=(33,15), dpi= 100, facecolor='w', edgecolor='k')

    labels = "123456789dodtcqssonv"
    groups = pd.DataFrame()

    for i, s in enumerate(model.cluster_centers_):

        grp_sites = a.columns[np.where(result == i)]
        groups = pd.concat([groups, 
                            pd.DataFrame({"group" : i, 
                                          "site": grp_sites})])
        groups.to_csv("data/Analysis/Clustering/"+str(j)+"_groups.csv", index=False)

        ax = plt.subplot(321+i)
        ax.plot(s.ravel(), color="r")

        cluster = q[np.where(result == i)]
        q_05 = np.quantile(cluster, 0.05, axis=0)
        q_95 = np.quantile(cluster, 0.95, axis=0)
        
        q_5 = np.quantile(cluster, 0.5, axis=0)
        median_serie = pd.concat([median_serie, 
                                  pd.DataFrame(data={"date":pd.date_range("1/1/2020",
                                                                          "28/11/2020",
                                                                          freq="D"
                                                                         ),
                                                     "values":q_5.ravel(),
                                                     "group":i
                                                    })], axis=0)

        ax.fill_between(np.arange(0, 333, 1), 
                        q_05.ravel(), q_95.ravel(), 
                        color="gray", alpha=0.3)

        ax.plot(q_5, color="blue")
        ax.set_title("Grupo "+str(i+1) + ": "+str(len(grp_sites))+" estaciones")

    fig.legend(handles=[mpatches.Patch(color='r', label='baricentro'),
                mpatches.Patch(color='b', label='Mediana')], loc="center right")
    fig.tight_layout()
    fig.savefig("Plots/"+str(j)+"_clusters_"+variable+".png")



FileNotFoundError: [Errno 2] No such file or directory: 'data/Analysis/Clustering/4_groups.csv'

<Figure size 3300x1500 with 0 Axes>

In [12]:
median_serie.to_csv("data/clusters_"+variable+".csv")